In [171]:
import pandas as pd
from sklearn.preprocessing import FunctionTransformer
import numpy as np

In [172]:
data = pd.read_csv("data_avec_FE.csv")

In [173]:
data

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,prix_euro,Brand,Model
0,maruti wagon r lxi cng,mumbai,2010,72000,cng,manual,first,26.60,998.0,58.16,5.0,NaN,1925.0,maruti,wagon
1,hyundai creta 1.6 crdi sx option,pune,2015,41000,diesel,manual,first,19.67,1582.0,126.20,5.0,NaN,13750.0,hyundai,creta
2,honda jazz v,chennai,2011,46000,petrol,manual,first,18.20,1199.0,88.70,5.0,8.61 lakh,4950.0,honda,jazz
3,maruti ertiga vdi,chennai,2012,87000,diesel,manual,first,20.77,1248.0,88.76,7.0,NaN,6600.0,maruti,ertiga
4,audi a4 new 2.0 tdi multitronic,coimbatore,2013,40670,diesel,automatic,second,15.20,1968.0,140.80,5.0,NaN,19514.0,audi,a4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5967,maruti swift vdi,delhi,2014,27365,diesel,manual,first,28.40,1248.0,74.00,5.0,7.88 lakh,5225.0,maruti,swift
5968,hyundai xcent 1.1 crdi s,jaipur,2015,100000,diesel,manual,first,24.40,1120.0,71.00,5.0,NaN,4400.0,hyundai,xcent
5969,mahindra xylo d4 bsiv,jaipur,2012,55000,diesel,manual,second,14.00,2498.0,112.00,8.0,NaN,3190.0,mahindra,xylo
5970,maruti wagon r vxi,kolkata,2013,46000,petrol,manual,first,18.90,998.0,67.10,5.0,NaN,2915.0,maruti,wagon


In [174]:
data.isna().sum()

Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                 143
Seats                  42
New_Price            5167
prix_euro               0
Brand                   0
Model                   0
dtype: int64

In [175]:
X = data.drop("prix_euro", axis=1)
y = data["prix_euro"]

In [176]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

In [177]:
def trouver_voitures_similaires(df):
    # Ajouter une colonne 'Similar_Cars' vide
    df['Similar_Cars'] = None
    
    for index, row in df.iterrows():
        marque_voiture = row['Brand']
        modele_voiture = row['Model']
        fuel_type = row['Fuel_Type']
        transmission = row['Transmission']
        
        # Rechercher des voitures ayant la même marque, modèle, carburant et transmission
        similar_cars = df[
            (df['Brand'] == marque_voiture) & 
            (df['Model'] == modele_voiture) & 
            (df['Fuel_Type'] == fuel_type) & 
            (df['Transmission'] == transmission)
        ]
        
        # Si au moins 2 véhicules (lui-même + un autre) sont trouvés, on ajoute les résultats
        if len(similar_cars) >= 2:
            df.at[index, 'Similar_Cars'] = ', '.join(similar_cars['Name'].values)
    
    return df

voitures_similaires = FunctionTransformer(trouver_voitures_similaires, validate=False) ## transformer pour mettre dans la pipeline 


In [178]:
def remplir_valeurs_manquantes(df):
    for index, row in df.iterrows():
        # Liste des colonnes à compléter
        list_colonnes = ['Mileage', 'Engine', 'Power', 'Seats']
        for col in list_colonnes:
            # Vérifier si la valeur dans la cellule est manquante (NaN)
            if pd.isnull(row[col]):  
                # Rechercher des véhicules similaires avec le même carburant et transmission
                similar_cars = df[df['Similar_Cars'].notnull()]  # Utiliser la colonne 'Similar_Cars' déjà remplie
                if not similar_cars.empty:
                    # Récupérer la valeur la plus fréquente (mode) pour la colonne en question
                    mode_value = similar_cars[col].mode()
                    if not mode_value.empty:  # Si un mode existe, utiliser la première valeur trouvée
                        df.at[index, col] = mode_value.iloc[0]  
                    else:  # Si aucun mode n'est trouvé, utiliser la médiane de la colonne entière
                        df.at[index, col] = df[col].median()  
                else:
                    # Si aucune voiture similaire n'est trouvée, remplir avec la médiane de la colonne
                    df.at[index, col] = df[col].median()  
    return df.drop(columns=["Similar_Cars"])

valeurs_manquantes = FunctionTransformer(remplir_valeurs_manquantes, validate=False) ## transformer pour mettre dans la pipeline 


In [179]:
def reduire_outliers(df):
    if df is None:
        raise ValueError("Le DataFrame fourni est None. Vérifiez la source de votre pipeline.")

    list_colonnes = ['Mileage', 'Engine', 'Power', 'Seats', 'Kilometers_Driven']
    list_colonnes = [col for col in list_colonnes if col in df.columns]  # Vérifie si les colonnes existent

    if not list_colonnes:
        print("Aucune colonne trouvée pour traiter les outliers.")
        return df  # Retourne le DataFrame inchangé

    df[list_colonnes] = df[list_colonnes].apply(pd.to_numeric, errors='coerce')

    Q1 = df[list_colonnes].quantile(0.25)
    Q3 = df[list_colonnes].quantile(0.75)
    IQR = Q3 - Q1
    borne_inf = Q1 - 1.5 * IQR
    borne_sup = Q3 + 1.5 * IQR

    df[list_colonnes] = df[list_colonnes].clip(lower=borne_inf, upper=borne_sup, axis=1)

    return df

outliers = FunctionTransformer(reduire_outliers, validate=False) ## transformer pour mettre dans la pipeline 

In [180]:
def drop_colonne(df):
    colonne_drop = ["New_Price", "Name"]
    colonne_drop = [col for col in colonne_drop if col in df.columns]
    df = df.drop(columns= colonne_drop, axis=1)
    return df

suprimer_colonne = FunctionTransformer(drop_colonne, validate=False)

In [182]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


def transformer(df):
    
    label = OrdinalEncoder()
    df[["Brand","Model"]] = label.fit_transform(df[["Brand","Model"]]) 
    


      
    numerical_cols = df.select_dtypes(include=np.number).columns
    categorical_cols_onehot = ["Fuel_Type","Transmission","Owner_Type"]


    processeur = ColumnTransformer([
        ("numerical", StandardScaler(), numerical_cols),
        ("categorical_cols_onehot", OneHotEncoder(sparse_output=False), categorical_cols_onehot)
    ])

    df_transformer = processeur.fit_transform(df)

    return df_transformer

transformers = FunctionTransformer(transformer, validate=False) ## transformer pour mettre dans la pipeline

# **Model**

In [253]:
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor, Lasso
from sklearn.feature_selection import RFECV, SelectKBest,  f_regression

In [254]:
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
dico_modeles = {"SVR":SVR(kernel="linear"),
                "KNeighborsRegressor":KNeighborsRegressor(),
                "LinearRegression":LinearRegression(),
                "RandomForestRegressor":RandomForestRegressor(),
                "SGDRegressor":SGDRegressor(),
                "Lasso":Lasso(),
                }

In [265]:
dico_modeles["SVR"]

SVR(kernel='linear')

In [266]:
Pipeline_modele = Pipeline([
    ("voitures_similaires",voitures_similaires),
    ("valeurs_manquantes",valeurs_manquantes),
    ("outliers",outliers),
    ("suprimer_colonne",suprimer_colonne),
    ("transformers", transformers),
])

# SVR

In [271]:
X_train_preprocessed = Pipeline_modele.fit_transform(X_train)
X_test_preprocessed = Pipeline_modele.transform(X_test)

modele = dico_modeles["SVR"]
        # Sélection des features avec RFECV
selector = RFECV(estimator=modele, step=1, cv=5, scoring="r2")  
selector.fit(X_train_preprocessed, y_train)

        # Transformation des données en ne gardant que les meilleures features
X_train_selected = selector.transform(X_train_preprocessed)
X_test_selected = selector.transform(X_test_preprocessed)

        # Entraînement du modèle avec les features sélectionnées
modele.fit(X_train_selected, y_train)
y_pred = modele.predict(X_test_selected)

        # Affichage des performances du modèle
print("Modèle utilisé :", modele.__class__.__name__)
print("R² :", r2_score(y_test, y_pred))
print("MSE :", mean_squared_error(y_test, y_pred))

Modèle utilisé : SVR
R² : 0.3813419040797823
MSE : 71794713.01951389


In [270]:
X_train_preprocessed = Pipeline_modele.fit_transform(X_train)
X_test_preprocessed = Pipeline_modele.transform(X_test)

modele = dico_modeles["LinearRegression"]
        # Sélection des features avec RFECV
selector = RFECV(estimator=modele, step=1, cv=5, scoring="r2")  
selector.fit(X_train_preprocessed, y_train)

        # Transformation des données en ne gardant que les meilleures features
X_train_selected = selector.transform(X_train_preprocessed)
X_test_selected = selector.transform(X_test_preprocessed)

        # Entraînement du modèle avec les features sélectionnées
modele.fit(X_train_selected, y_train)
y_pred = modele.predict(X_test_selected)

        # Affichage des performances du modèle
print("Modèle utilisé :", modele.__class__.__name__)
print("R² :", r2_score(y_test, y_pred))
print("MSE :", mean_squared_error(y_test, y_pred))

Modèle utilisé : LinearRegression
R² : 0.6937710385833904
MSE : 35537594.26758483


In [272]:
X_train_preprocessed = Pipeline_modele.fit_transform(X_train)
X_test_preprocessed = Pipeline_modele.transform(X_test)

modele = dico_modeles["SGDRegressor"]
        # Sélection des features avec RFECV
selector = RFECV(estimator=modele, step=1, cv=5, scoring="r2")  
selector.fit(X_train_preprocessed, y_train)

        # Transformation des données en ne gardant que les meilleures features
X_train_selected = selector.transform(X_train_preprocessed)
X_test_selected = selector.transform(X_test_preprocessed)

        # Entraînement du modèle avec les features sélectionnées
modele.fit(X_train_selected, y_train)
y_pred = modele.predict(X_test_selected)

        # Affichage des performances du modèle
print("Modèle utilisé :", modele.__class__.__name__)
print("R² :", r2_score(y_test, y_pred))
print("MSE :", mean_squared_error(y_test, y_pred))

Modèle utilisé : SGDRegressor
R² : 0.6918688438753933
MSE : 35758342.2446476


In [273]:
X_train_preprocessed = Pipeline_modele.fit_transform(X_train)
X_test_preprocessed = Pipeline_modele.transform(X_test)

modele = dico_modeles["Lasso"]
        # Sélection des features avec RFECV
selector = RFECV(estimator=modele, step=1, cv=5, scoring="r2")  
selector.fit(X_train_preprocessed, y_train)

        # Transformation des données en ne gardant que les meilleures features
X_train_selected = selector.transform(X_train_preprocessed)
X_test_selected = selector.transform(X_test_preprocessed)

        # Entraînement du modèle avec les features sélectionnées
modele.fit(X_train_selected, y_train)
y_pred = modele.predict(X_test_selected)

        # Affichage des performances du modèle
print("Modèle utilisé :", modele.__class__.__name__)
print("R² :", r2_score(y_test, y_pred))
print("MSE :", mean_squared_error(y_test, y_pred))

Modèle utilisé : Lasso
R² : 0.693570792774774
MSE : 35560832.61273506


In [275]:
X_train_preprocessed = Pipeline_modele.fit_transform(X_train)
X_test_preprocessed = Pipeline_modele.transform(X_test)

modele = dico_modeles["RandomForestRegressor"]
        # Sélection des features avec RFECV
selector = RFECV(estimator=modele, step=1, cv=5, scoring="r2")  
selector.fit(X_train_preprocessed, y_train)

        # Transformation des données en ne gardant que les meilleures features
X_train_selected = selector.transform(X_train_preprocessed)
X_test_selected = selector.transform(X_test_preprocessed)

        # Entraînement du modèle avec les features sélectionnées
modele.fit(X_train_selected, y_train)
y_pred = modele.predict(X_test_selected)

        # Affichage des performances du modèle
print("Modèle utilisé :", modele.__class__.__name__)
print("R² :", r2_score(y_test, y_pred))
print("MSE :", mean_squared_error(y_test, y_pred))

Modèle utilisé : RandomForestRegressor
R² : 0.8471942757811377
MSE : 17732966.222181268
